This notebook is used to gather the data about the games played during the World 2018

To access to this data, we rely on the lolesports.com website, and some undocumented endpoints it uses. The data from these endpoints is quite messy and it can be tricky to get to what we want.

# 1 - Get the matches

We will get started in your data collection by finding all the matches of the tournament.

The first endpoint we will use is the one giving data about the tournaments. More precisely, tournament with leagueId 9 is the one about Worlds since 2015.

In [38]:
import requests, json

urlTournament="http://api.lolesports.com/api/v1/scheduleItems?leagueId=9"
r  = requests.get(urlTournament)
rawTournamentData = json.loads(r.text)

In [39]:
rawTournamentData.keys()

dict_keys(['teams', 'highlanderTournaments', 'scheduleItems', 'players', 'highlanderRecords'])

The response for this endpoint has multiple fields, and the one that interests us is the "highlanderTournaments" containing a list of all instances of the tournament, here the 4 Worlds championships since 2015

In [40]:
rawTournamentData["highlanderTournaments"][3].keys()

dict_keys(['leagueId', 'league', 'platformIds', 'breakpoints', 'gameIds', 'endDate', 'brackets', 'roles', 'title', 'description', 'startDate', 'id', 'leagueReference', 'rosters', 'liveMatches', 'published', 'queues'])

The "id" field of a tournament will be usefull later, we need to save it.

In [41]:
tournamentId = rawTournamentData['highlanderTournaments'][3]["id"]

Among a lot of data, the "brackets" field is the one we want. The tournament follows this structure : 
 - A tournament is a list of brackets, usually more or less independant. It can be a group from the group phase (each group has its own bracket) or a whole phase like the elimination (the tree with final, semi-finals...)
 - A bracket is a list of matches
 - A match is a confrontation between two teams, and handles the "Best Of" format (BO2, BO3, BO5...). So a match is a list of games.

From now, we can parse all games of all matches of all brackets. We will need two specific informations : the gameId for the match history, the gameId for lolesports which we will call gameUuid, and the matchId

In [42]:
#Get the bracket for Worlds 2018
brackets = rawTournamentData['highlanderTournaments'][3]["brackets"]

matches = []
games = {}

#Get the list of matches and each of their games
for bracketId in brackets:
    bracket = brackets[bracketId]
    
    for matchId in bracket["matches"]:
        match = bracket["matches"][matchId]
        
        for gameUuid in match['games']:
            
            game = match['games'][gameUuid]
            
            if 'gameId' in game:
                matches.append(matchId)
                games[gameUuid] = {"matchHistoryId":game['gameId']}

# 2 - Get the hashes

Now we have some gameId, we could use this : 

In [43]:
games[next(iter(games.keys()))]

{'matchHistoryId': '2002860215'}

to go to the matchHistory website and have all we want by folowing this link : 

In [44]:
baseMatchHistoryUrl = "https://matchhistory.na.leagueoflegends.com/en/#match-details/TRLH2/"
baseMatchHistoryUrl + games[next(iter(games.keys()))]["matchHistoryId"]

'https://matchhistory.na.leagueoflegends.com/en/#match-details/TRLH2/2002860215'

However, the match history pages are protected by a hash.

We can obtain the hashes by using a second lolesports endpoint serving matches details, which will need the tournamentId we saved earlier

In [45]:
matchId = next(iter(games.keys()))
baseMatchUrl = "http://api.lolesports.com/api/v2/highlanderMatchDetails?tournamentId="+tournamentId+"&matchId="
r  = requests.get(baseMatchUrl + matches[0])
matchData = json.loads(r.text)

This give some data about the teams, the links to videos of the games, and more important for us, the games hashes. We will be able to link games of the match to their respective hash

In [46]:
for i in matchData["gameIdMappings"]:
    games[i["id"]]["hash"] = i["gameHash"]

Now repeat the process for every match

In [49]:
for matchId in matches:
    r  = requests.get(baseMatchUrl + matchId)
    matchData = json.loads(r.text)
    for i in matchData["gameIdMappings"]:
        games[i["id"]]["hash"] = i["gameHash"]

And access to the match history page

In [51]:
baseMatchHistoryUrl + games[next(iter(games.keys()))]["matchHistoryId"] + "?gameHash="+ games[next(iter(games.keys()))]["hash"]

'https://matchhistory.na.leagueoflegends.com/en/#match-details/TRLH2/2002860215?gameHash=025021dc6eefd5b8'

# 3 - Get the games data

We now have access to the game data. Like the Riot Games API, game data is split in to endpoint, one for the general data, and one about the timeline data.

In [54]:
baseMatchHistoryStatsUrl = "https://acs.leagueoflegends.com/v1/stats/game/TRLH2/"
baseMatchHistoryStatsUrl + games[next(iter(games.keys()))]["matchHistoryId"] + "?gameHash="+ games[next(iter(games.keys()))]["hash"]

'https://acs.leagueoflegends.com/v1/stats/game/TRLH2/2002860215?gameHash=025021dc6eefd5b8'

In [56]:
baseMatchHistoryStatsUrl + games[next(iter(games.keys()))]["matchHistoryId"] + "/timeline?gameHash="+ games[next(iter(games.keys()))]["hash"]

'https://acs.leagueoflegends.com/v1/stats/game/TRLH2/2002860215/timeline?gameHash=025021dc6eefd5b8'